In [29]:
import torch
from torch import nn

from torch.optim import lr_scheduler

from torch.utils.data import random_split,Dataset,DataLoader

import torch.nn.functional as F
import torch.nn.init as init

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import time
import copy
import random
import pickle
import tarfile

import argparse
import pandas as pd
import dgl
import torch
import torch.nn.functional as F
import collections
from scipy.sparse import csr_matrix, vstack, save_npz
from sklearn.decomposition import PCA
from pathlib import Path
import numpy as np
from pprint import pprint
import json
from collections import Counter

import time


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [30]:
import scanpy as sc
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.model_selection import train_test_split

In [31]:
from dgl.contrib.sampling import NeighborSampler

In [32]:
import dgl.function as fn

In [33]:
torch.version

<module 'torch.version' from '/home/chenhuaguan/.conda/envs/py36/lib/python3.6/site-packages/torch/version.py'>

# Create Graph

In [34]:
device = torch.device('cpu')

In [35]:
adata = sc.read("data/Limb_Muscle_facts_processed_3m.h5ad", dtype='float64')

In [36]:
data_gene = adata.X

In [37]:
data_gene = pd.DataFrame(data_gene.T)

In [38]:
id2gene = data_gene.index.values.tolist()
id2gene.sort()

In [39]:
data_cell = pd.DataFrame(adata.obs["cell_ontology_class"]).reset_index()
data_cell.columns = ['Cell','Cell_type']
data_cell.index = data_cell.index+1

In [40]:
cell = set()
cell_type_list = []
data_cell['Cell_type'] = data_cell['Cell_type'].map(str.strip)
cell_types = set(data_cell.values[:, 1])
cell_type_list.extend(data_cell.values[:, 1].tolist())
id2label = list(cell_types)
label_statistics = dict(collections.Counter(cell_type_list))
total_cell = sum(label_statistics.values())

In [41]:
for label, num in label_statistics.items():
    if num / total_cell <= 0.005:
        id2label.remove(label)  # remove exclusive labels
gene2id = {gene: idx for idx, gene in enumerate(id2gene)}
num_genes = len(id2gene)
# prepare unified labels
num_labels = len(id2label)
label2id = {label: idx for idx, label in enumerate(id2label)}

In [42]:
graph = dgl.DGLGraph()

gene_ids = torch.arange(num_genes, dtype=torch.int32, device=device).unsqueeze(-1)
graph.add_nodes(num_genes, {'id': gene_ids})
all_labels = []
matrices = []
num_cells = 0
cell2type = pd.DataFrame(adata.obs["cell_ontology_class"]).reset_index()
cell2type.index = cell2type.index+1
cell2type.columns = ['cell', 'type']
cell2type['type'] = cell2type['type'].map(str.strip)
cell2type['id'] = cell2type['type'].map(label2id)
filter_cell = np.where(pd.isnull(cell2type['id']) == False)[0]
cell2type = cell2type.iloc[filter_cell]
all_labels += cell2type['id'].tolist()
df = pd.DataFrame(adata.X)

In [43]:
df = df.iloc[filter_cell]
df = df.rename(columns=gene2id)
col = [c for c in df.columns if c in gene2id.values()]
df = df[col]
arr = df.to_numpy()
row_idx, col_idx = np.nonzero(arr > 0)  # intra-dataset index
non_zeros = arr[(row_idx, col_idx)]  # non-zero values
cell_idx = row_idx + graph.number_of_nodes()  # cell_index
gene_idx = df.columns[col_idx].astype(int).tolist()  # gene_index
info_shape = (len(df), num_genes)
info = csr_matrix((non_zeros, (row_idx, gene_idx)), shape=info_shape)
matrices.append(info)

num_cells += len(df)

In [44]:
ids = torch.tensor([-1] * len(df), dtype=torch.int32, device=device).unsqueeze(-1)
graph.add_nodes(len(df), {'id': ids})
graph.add_edges(cell_idx, gene_idx,
                {'weight': torch.tensor(non_zeros, dtype=torch.float32, device=device).unsqueeze(1)})
graph.add_edges(gene_idx, cell_idx,
                {'weight': torch.tensor(non_zeros, dtype=torch.float32, device=device).unsqueeze(1)})

In [45]:
sparse_feat = vstack(matrices).toarray()  # cell-wise  (cell, gene)
gene_pca = PCA(400, random_state=10086).fit(sparse_feat.T)
gene_feat = gene_pca.transform(sparse_feat.T)
gene_evr = sum(gene_pca.explained_variance_ratio_) * 100
sparse_feat = sparse_feat / (np.sum(sparse_feat, axis=1, keepdims=True) + 1e-6)
# use weighted gene_feat as cell_feat
cell_feat = sparse_feat.dot(gene_feat)
gene_feat = torch.from_numpy(gene_feat)  # use shared storage
cell_feat = torch.from_numpy(cell_feat)

graph.ndata['features'] = torch.cat([gene_feat, cell_feat], dim=0).type(torch.float).to(device)
labels = torch.tensor([-1] * num_genes + all_labels, dtype=torch.long, device=device)  # [gene_num+train_num]
per = np.random.permutation(range(num_genes, num_genes + num_cells))
test_ids = torch.tensor(per[:int(num_cells // ((1 - 0.2) / 0.2 + 1))]).to(device)
train_ids = torch.tensor(per[int(num_cells // ((1 - 0.2) / 0.2 + 1)):]).to(device)

In [46]:
in_degrees = graph.in_degrees()
for i in range(graph.number_of_nodes()):
    src, dst, in_edge_id = graph.in_edges(i, form='all')
    if src.shape[0] == 0:
        continue
    edge_w = graph.edata['weight'][in_edge_id]
    graph.edata['weight'][in_edge_id] = in_degrees[i] * edge_w / torch.sum(edge_w)

In [47]:
graph.add_edges(graph.nodes(), graph.nodes(),
                {'weight': torch.ones(graph.number_of_nodes(), dtype=torch.float, device=device).unsqueeze(1)})
graph.readonly(True)

# GNN

## NodeUpdate

In [48]:
class NodeUpdate(nn.Module):
    def __init__(self, in_feats, out_feats, activation=None, norm=None):
        super(NodeUpdate, self).__init__()
        self.fc_neigh = nn.Linear(in_features=in_feats, out_features=out_feats)
        self.activation = activation
        self.norm = norm
        nn.init.xavier_uniform_(self.fc_neigh.weight, gain=nn.init.calculate_gain('relu'))

    def forward(self, node):
        h_neigh = node.data['neigh']
        h_neigh = self.fc_neigh(h_neigh)
        if self.activation is not None:
            h_neigh = self.activation(h_neigh)
        if self.norm is not None:
            h_neigh = self.norm(h_neigh)
        return {'activation': h_neigh}

## GNN Model

In [49]:
class GNN(nn.Module):
    def __init__(self, in_feats, n_hidden, n_classes, n_layers, gene_num, activation=None, norm=None, dropout=0.0):
        super(GNN, self).__init__()
        self.n_layers = n_layers
        self.gene_num = gene_num
        if dropout != 0:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = None
        self.layers = nn.ModuleList()
        self.layers.append(NodeUpdate(in_feats=in_feats, out_feats=n_hidden, activation=activation, norm=norm))
        for _ in range(n_layers - 1):
            self.layers.append(NodeUpdate(in_feats=n_hidden, out_feats=n_hidden, activation=activation, norm=norm))

        # [gene_num] is alpha of gene-gene, [gene_num+1] is alpha of cell-cell self loop
        self.alpha = nn.Parameter(torch.tensor([1] * (self.gene_num + 2), dtype=torch.float32).unsqueeze(-1))
        self.linear = nn.Linear(n_hidden, n_classes)
        nn.init.xavier_uniform_(self.linear.weight, gain=nn.init.calculate_gain('relu'))

    def message_func(self, edges: dgl.udf.EdgeBatch):
        number_of_edges = edges.src['h'].shape[0]
        indices = np.expand_dims(np.array([self.gene_num + 1] * number_of_edges, dtype=np.int32), axis=1)
        src_id, dst_id = edges.src['id'].cpu().numpy(), edges.dst['id'].cpu().numpy()
        indices = np.where((src_id >= 0) & (dst_id < 0), src_id, indices)  # gene->cell
        indices = np.where((dst_id >= 0) & (src_id < 0), dst_id, indices)  # cell->gene
        indices = np.where((dst_id >= 0) & (src_id >= 0), self.gene_num, indices)  # gene-gene
        h = edges.src['h'] * self.alpha[indices.squeeze()]
        # return {'m': h}
        return {'m': h * edges.data['weight']}

    def forward(self, nf: dgl.NodeFlow):
        nf.layers[0].data['activation'] = nf.layers[0].data['features']
        for i, layer in enumerate(self.layers):
            h = nf.layers[i].data.pop('activation')
            if self.dropout:
                h = self.dropout(h)
            nf.layers[i].data['h'] = h
            nf.block_compute(i, self.message_func, fn.mean('m', 'neigh'), layer)
        h = nf.layers[-1].data.pop('activation')
        h = self.linear(h)
        return h

    def evaluate(self, nf: dgl.NodeFlow):
        def message_func(edges: dgl.EdgeBatch):
            # edges.src['h']： (number of edges, feature dim)
            number_of_edges = edges.src['h'].shape[0]
            indices = np.expand_dims(np.array([self.gene_num + 1] * number_of_edges, dtype=np.int32), axis=1)
            src_id, dst_id = edges.src['id'].cpu().numpy(), edges.dst['id'].cpu().numpy()
            indices = np.where((src_id >= 0) & (dst_id < 0), src_id, indices)  # gene->cell
            indices = np.where((dst_id >= 0) & (src_id < 0), dst_id, indices)  # cell->gene
            indices = np.where((dst_id >= 0) & (src_id >= 0), self.gene_num, indices)  # gene-gene
            h = edges.src['h'].cpu() * self.alpha[indices.squeeze()]
            return {'m': h * edges.data['weight'].cpu()}

        nf.layers[0].data['activation'] = nf.layers[0].data['features'].cpu()
        for i, layer in enumerate(self.layers):
            h = nf.layers[i].data.pop('activation')
            if self.dropout:
                h = self.dropout(h)
            nf.layers[i].data['h'] = h
            nf.block_compute(i, message_func, fn.mean('m', 'neigh'), layer)
        h = nf.layers[-1].data.pop('activation')
        h = self.linear(h)
        return h

# Adam_Train

In [50]:
class Adam_Trainer:
    def __init__(self, device_train,gnn_model,lr,weight_decay,Adam_epochs,batch_size,
                num_cells,
                num_genes,
                num_labels,
                graph,
                train_ids,
                test_ids,
                labels):
        
        self.device = device_train
        
        self.num_cells = num_cells
        self.num_genes = num_genes
        self.num_labels = num_labels
        
        self.graph = graph
        
        self.graph.readonly(True)
        
        self.train_ids = train_ids
        self.test_ids = test_ids
        self.labels = labels 
        
        self.labels = self.labels.to(self.device)
        
        self.model = gnn_model
        
        self.lr = lr
        
        self.weight_decay = weight_decay
        
        self.epochs = Adam_epochs
        
        self.batch_size = batch_size

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr,
                                          weight_decay=self.weight_decay)
        
        self.loss_fn = nn.CrossEntropyLoss(reduction='sum')
        
        self.num_neighbors = self.num_cells + self.num_genes

    def fit(self):
        max_test_acc, _train_acc, _epoch = 0, 0, 0
        for epoch in range(self.epochs):
            loss = self.train()
            train_correct, train_unsure = self.evaluate(self.train_ids, 'train')
            train_acc = train_correct / len(self.train_ids)
            test_correct, test_unsure = self.evaluate(self.test_ids, 'test')
            test_acc = test_correct / len(self.test_ids)
            if max_test_acc <= test_acc:
                final_test_correct_num = test_correct
                final_test_unsure_num = test_unsure
                _train_acc = train_acc
                _epoch = epoch
                max_test_acc = test_acc
                self.save_model()
            print(
                f">>>>Epoch {epoch+1:04d}: Train Acc {train_acc:.4f}, Loss {loss / len(self.train_ids):.4f}, Test correct {test_correct}, "
                f"Test unsure {test_unsure}, Test Acc {test_acc:.4f}")
            if train_acc == 1:
                break

        #print(f"---{self.params.species} {self.params.tissue} Best test result:---")
        print(f"Epoch {_epoch+1:04d}, Train Acc {_train_acc:.4f}, Test Correct Num {final_test_correct_num}, Test Total Num {len(self.test_ids)}, Test Unsure Num {final_test_unsure_num}, Test Acc {final_test_correct_num / len(self.test_ids):.4f}")

    def train(self):
        self.model.train()
        total_loss = 0
        for batch, nf in enumerate(NeighborSampler(g=self.graph,
                                                   batch_size=self.batch_size,
                                                   expand_factor=self.num_neighbors,
                                                   num_hops=1,
                                                   neighbor_type='in',
                                                   shuffle=True,
                                                   num_workers=8,
                                                   seed_nodes=self.train_ids.long())):
            nf.copy_from_parent()  # Copy node/edge features from the parent graph.
            logits = self.model(nf)
            batch_nids = nf.layer_parent_nid(-1).type(torch.long).to(device=self.device)
            loss = self.loss_fn(logits, self.labels[batch_nids])
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()

        return total_loss

    def evaluate(self, ids, type='test'):
        self.model.eval()
        total_correct, total_unsure = 0, 0
        for nf in NeighborSampler(g=self.graph,
                                  batch_size=self.batch_size,
                                  expand_factor=self.num_cells + self.num_genes,
                                  num_hops=1,
                                  neighbor_type='in',
                                  shuffle=True,
                                  num_workers=8,
                                  seed_nodes=ids.long()):
            nf.copy_from_parent()  # Copy node/edge features from the parent graph.
            with torch.no_grad():
                logits = self.model(nf).cpu()
            batch_nids = nf.layer_parent_nid(-1).type(torch.long)
            logits = nn.functional.softmax(logits, dim=1).numpy()
            label_list = self.labels.cpu()[batch_nids]
            for pred, label in zip(logits, label_list):
                max_prob = pred.max().item()
                if max_prob < 2 / num_labels:
                    total_unsure += 1
                elif pred.argmax().item() == label:
                    total_correct += 1

        return total_correct, total_unsure

    def save_model(self):
        state = {
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict()
        }

        torch.save(state, "GA_Model.pth")

# GA

In [51]:
def crossover_and_mutation(parents, sigma=0.01):

    
    base_sd = parents[0].state_dict()
    keys = base_sd                    # use all layers to be affected
    
    # Sum of the weights of the parent
    for i in range(1, len(parents)):
        parent_sd = parents[i].state_dict()
        for key in keys:
            base_sd[key] = base_sd[key] + parent_sd[key]
            
    
    # Average and add mutation
    num_parents = len(parents)
    
    for key in keys:
        
        tensor_size = base_sd[key].size()
        random_tensor = torch.normal(mean=0.0, std=sigma, size=tensor_size)
        
        base_sd[key] = (base_sd[key] / num_parents) + random_tensor
    
    # create offspring
    offspring = GNN(in_feats=in_feats,
                         n_hidden=n_hidden,
                         n_classes=num_labels,
                         n_layers=1,
                         gene_num=num_genes,
                         activation=F.relu,
                         dropout=0.1).to(device_train)
    
    offspring.load_state_dict(base_sd)
    
    return offspring
    

def create_offspring(population, fitness, rho, sigma):

    
    # Perform selection
    parents = random.choices(population,weights=fitness, k=rho) 
    
    # Perform crossover and mutation
    offspring = crossover_and_mutation(parents, sigma)
    
    
    return offspring


def GA_training(population, pop_size, offspring_size, elitist_level, rho, sigma, train_ids,
                test_ids,
                graph,
                batch_size,
                num_cells,
                num_genes,
                labels):
    
    #Calculate fitness of trained population

    fitness = [calc_loss(population[i],train_ids,graph,batch_size,num_cells,num_genes,labels) for i in range(pop_size)]
    
    print(f"--- -- Finished fitness evaluation, length: {len(fitness)}")
    
    #Create offspring population
    
    fitness_weighted = fitness
    offspring_population = [create_offspring(population, fitness_weighted, rho, sigma) for i in range(offspring_size)]
    
    print("--- -- Finished creating offspring population")
    
    #Evaluate fitness of offsprings 
    
    offspring_fitness = [calc_loss(offspring_population[i],test_ids,graph,batch_size,num_cells,num_genes,labels) for i in range(offspring_size)]
    
    print("--- -- Finished evaluating fitness of offspring population")
    
    # Combine fitness and population lists
    
    combined_fitness = fitness + offspring_fitness
    combined_population = population + offspring_population
    
    # sort and select population by their fitness values
    
    sorted_population = [pop for _, pop in sorted(zip(combined_fitness, combined_population), key=lambda pair: pair[0])]
    sorted_fitness = [loss for loss, _ in sorted(zip(combined_fitness, combined_population), key=lambda pair: pair[0])]
    
    m = int(pop_size * elitist_level)
    new_population = sorted_population[0:m]
    
    # Fill up rest of population
    difference = pop_size - m
    remaining_population = list(set(sorted_population) - set(new_population))
    filler_population = random.sample(remaining_population, difference)
    
    # assemble new population and return
    new_population = new_population + filler_population
    
    return new_population, sorted_fitness

In [52]:
def calc_loss(model, train_ids,graph,batch_size,num_cells,num_genes,labels):
    model.eval()
    total_correct, total_unsure = 0, 0
    for nf in NeighborSampler(g=graph,
                              batch_size=batch_size,
                              expand_factor=num_cells + num_genes,
                              num_hops=1,
                              neighbor_type='in',
                              shuffle=True,
                              num_workers=8,
                              seed_nodes=train_ids.long()):
        nf.copy_from_parent()  # Copy node/edge features from the parent graph.
        with torch.no_grad():
            logits = model(nf).cpu()
        batch_nids = nf.layer_parent_nid(-1).type(torch.long)
        logits = nn.functional.softmax(logits, dim=1).numpy()
        label_list = labels.cpu()[batch_nids]
        for pred, label in zip(logits, label_list):
            max_prob = pred.max().item()
            if max_prob < 2 / num_labels:
                total_unsure += 1
            elif pred.argmax().item() == label:
                total_correct += 1

    return total_correct

# GA_Neural

In [53]:
def GA_Neural_train(population,
                    pop_size,
                    max_generations, 
                    Adam_epochs, GA_steps, 
                    offspring_size, elitist_level, rho,
                    learning_rate,
                   weight_decay,
                   batch_size,
                   num_cells,num_genes,graph,train_ids,test_ids,labels):
    
    print(f"Starting with population of size: {pop_size}")
    
    
    for k in range(max_generations):
        print(f"Currently in generation {k+1}")
        
        #SGD
        print(f"--- Starting Adam")
        
        # Sequential version
        #population = [SGD_training(population[i], SGD_steps, learning_rate, 0.9, train_loader) for i in range(pop_size)]
        for i in range(pop_size):
            train = Adam_Trainer(device_train = device_train,gnn_model = population[i],
                                 lr = learning_rate,weight_decay = weight_decay,Adam_epochs = Adam_epochs,
                                 batch_size = batch_size,num_cells = num_cells,
                                                        num_genes = num_genes,
                                                        num_labels = num_labels,
                                                        graph = graph,
                                                        train_ids = train_ids,
                                                        test_ids = test_ids,
                                                        labels = labels)
            train.fit()
        
        print(f"--- Finished Adam")
         
        # GA
        print(f"--- Starting GA")
        GA_start = time.time()
        sorted_fitness = []          # store the sorted fitness values to maybe use in data collection
        for i in range(0, GA_steps):
            
            sigma = 0.01 / (k+1)
            population, sorted_fitness = GA_training(population, pop_size, offspring_size, elitist_level, rho, sigma, train_ids,
                                                    test_ids,
                                                    graph,
                                                    batch_size,
                                                    num_cells,
                                                    num_genes,
                                                    labels)
        
        GA_end = time.time()
        print(f"--- Finished GA,Time:{(GA_end-GA_start)*1000}ms")
        
        
    print(f"Finished training process")
    return population

# Start training process
We have now defined the whole training algorithm. The next step is to actually perform training.

In [57]:
# Hyperparameters
#device_train = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device_train = torch.device('cpu')
batch_size = 500
pop_size = 10
max_generations = 10
Adam_epochs = 10
GA_steps = 1
offspring_size = 80
elitist_level = 0.6
rho = 3
learning_rate = 1e-3
in_feats = 400
n_hidden = 200
weight_decay = 5e-4

In [58]:
# Create population and start training process
population = [GNN(in_feats=in_feats,
                         n_hidden=n_hidden,
                         n_classes=num_labels,
                         n_layers=1,
                         gene_num=num_genes,
                         activation=F.relu,
                         dropout=0.1).to(device_train) for i in range(pop_size)]

# Train Process

In [59]:
Train_start = time.time()
trained_population = GA_Neural_train(population=population,
                                    pop_size = pop_size,
                                    max_generations=max_generations,
                                    Adam_epochs=Adam_epochs,GA_steps=GA_steps,
                                    offspring_size=offspring_size,elitist_level=elitist_level,rho=rho,
                                    learning_rate=learning_rate,
                               weight_decay = weight_decay,
                               batch_size = batch_size,
                               num_cells = num_cells,num_genes = num_genes,graph = graph,
                                     train_ids = train_ids,test_ids = test_ids,
                                     labels = labels)
Train_end = time.time()
print(f"All Time:{(Train_start-Train_end)*1000}ms")

Starting with population of size: 10
Currently in generation 1
--- Starting Adam
>>>>Epoch 0001: Train Acc 0.4932, Loss 1.4025, Test correct 112, Test unsure 27, Test Acc 0.5091
>>>>Epoch 0002: Train Acc 0.5283, Loss 1.1339, Test correct 118, Test unsure 27, Test Acc 0.5364
>>>>Epoch 0003: Train Acc 0.6712, Loss 0.9384, Test correct 133, Test unsure 33, Test Acc 0.6045
>>>>Epoch 0004: Train Acc 0.7755, Loss 0.7745, Test correct 157, Test unsure 29, Test Acc 0.7136
>>>>Epoch 0005: Train Acc 0.8209, Loss 0.6549, Test correct 167, Test unsure 31, Test Acc 0.7591
>>>>Epoch 0006: Train Acc 0.8435, Loss 0.5622, Test correct 174, Test unsure 28, Test Acc 0.7909
>>>>Epoch 0007: Train Acc 0.8594, Loss 0.4854, Test correct 178, Test unsure 28, Test Acc 0.8091
>>>>Epoch 0008: Train Acc 0.8934, Loss 0.4117, Test correct 184, Test unsure 23, Test Acc 0.8364
>>>>Epoch 0009: Train Acc 0.9184, Loss 0.3596, Test correct 189, Test unsure 22, Test Acc 0.8591
>>>>Epoch 0010: Train Acc 0.9376, Loss 0.3047,

>>>>Epoch 0008: Train Acc 0.8787, Loss 0.5309, Test correct 181, Test unsure 21, Test Acc 0.8227
>>>>Epoch 0009: Train Acc 0.8946, Loss 0.4631, Test correct 186, Test unsure 20, Test Acc 0.8455
>>>>Epoch 0010: Train Acc 0.9116, Loss 0.4000, Test correct 190, Test unsure 17, Test Acc 0.8636
Epoch 0010, Train Acc 0.9116, Test Correct Num 190, Test Total Num 220, Test Unsure Num 17, Test Acc 0.8636
>>>>Epoch 0001: Train Acc 0.0964, Loss 1.8826, Test correct 19, Test unsure 47, Test Acc 0.0864
>>>>Epoch 0002: Train Acc 0.4422, Loss 1.5424, Test correct 93, Test unsure 34, Test Acc 0.4227
>>>>Epoch 0003: Train Acc 0.4887, Loss 1.2912, Test correct 110, Test unsure 30, Test Acc 0.5000
>>>>Epoch 0004: Train Acc 0.4977, Loss 1.1105, Test correct 111, Test unsure 28, Test Acc 0.5045
>>>>Epoch 0005: Train Acc 0.5431, Loss 0.9755, Test correct 117, Test unsure 22, Test Acc 0.5318
>>>>Epoch 0006: Train Acc 0.5998, Loss 0.8472, Test correct 125, Test unsure 21, Test Acc 0.5682
>>>>Epoch 0007: Train

>>>>Epoch 0001: Train Acc 0.7234, Loss 1.1887, Test correct 145, Test unsure 75, Test Acc 0.6591
>>>>Epoch 0002: Train Acc 0.7823, Loss 0.9907, Test correct 159, Test unsure 60, Test Acc 0.7227
>>>>Epoch 0003: Train Acc 0.8095, Loss 0.8348, Test correct 161, Test unsure 56, Test Acc 0.7318
>>>>Epoch 0004: Train Acc 0.8322, Loss 0.7123, Test correct 164, Test unsure 50, Test Acc 0.7455
>>>>Epoch 0005: Train Acc 0.8469, Loss 0.6174, Test correct 171, Test unsure 40, Test Acc 0.7773
>>>>Epoch 0006: Train Acc 0.8673, Loss 0.5241, Test correct 177, Test unsure 33, Test Acc 0.8045
>>>>Epoch 0007: Train Acc 0.8878, Loss 0.4704, Test correct 181, Test unsure 29, Test Acc 0.8227
>>>>Epoch 0008: Train Acc 0.9025, Loss 0.4114, Test correct 185, Test unsure 24, Test Acc 0.8409
>>>>Epoch 0009: Train Acc 0.9218, Loss 0.3590, Test correct 193, Test unsure 18, Test Acc 0.8773
>>>>Epoch 0010: Train Acc 0.9410, Loss 0.3214, Test correct 199, Test unsure 15, Test Acc 0.9045
Epoch 0010, Train Acc 0.9410, 

>>>>Epoch 0005: Train Acc 0.8991, Loss 0.4736, Test correct 177, Test unsure 33, Test Acc 0.8045
>>>>Epoch 0006: Train Acc 0.9138, Loss 0.4217, Test correct 186, Test unsure 26, Test Acc 0.8455
>>>>Epoch 0007: Train Acc 0.9331, Loss 0.3712, Test correct 197, Test unsure 16, Test Acc 0.8955
>>>>Epoch 0008: Train Acc 0.9467, Loss 0.3318, Test correct 200, Test unsure 14, Test Acc 0.9091
>>>>Epoch 0009: Train Acc 0.9535, Loss 0.2925, Test correct 203, Test unsure 12, Test Acc 0.9227
>>>>Epoch 0010: Train Acc 0.9683, Loss 0.2530, Test correct 204, Test unsure 12, Test Acc 0.9273
Epoch 0010, Train Acc 0.9683, Test Correct Num 204, Test Total Num 220, Test Unsure Num 12, Test Acc 0.9273
>>>>Epoch 0001: Train Acc 0.8073, Loss 0.8575, Test correct 160, Test unsure 57, Test Acc 0.7273
>>>>Epoch 0002: Train Acc 0.8118, Loss 0.7602, Test correct 161, Test unsure 49, Test Acc 0.7318
>>>>Epoch 0003: Train Acc 0.8254, Loss 0.6794, Test correct 162, Test unsure 45, Test Acc 0.7364
>>>>Epoch 0004: Tra

KeyboardInterrupt: 